In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets.widgets import Layout, HBox, VBox
import bqplot as bqp 

In [2]:
data_df = pd.read_csv('data.csv')

In [3]:
temp_df = data_df[~data_df["YearFrom"].isin(["16..","17.."])]  

In [4]:
temp_df = temp_df.astype({"YearFrom": float})
# temp_df["YearFrom"] = pd.to_numeric(temp_df["YearFrom"])

In [5]:
column=widgets.Dropdown(
    options=["Age", "VerdachteAchternaam/AccusedLastName","Suspect Professional/AccusedProfession", "Misdrijven/Crimes"],
    value='VerdachteAchternaam/AccusedLastName',
    description='Column:')

years=widgets.IntRangeSlider(
    value=[1600, 1700],
    min=1500, max=1803, step=1,
    description='Select Year:',
    continuous_update=False)

b_refresh = widgets.Button(
    description='Refresh',
    icon='fa-refresh',
    button_style='warning',
    layout=Layout(width='100px',
    margin = '0px 0px 0px 60px')
)

x_ord = bqp.OrdinalScale()
y_sc = bqp.LinearScale()

df=temp_df[column.value].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')


Bar = bqp.Bars(
    x = df['unique_values'],
    y = df['counts'],
    scales={'x': x_ord, 'y': y_sc}, 
    orientation='horizontal'
)


x_ax = bqp.Axis(
    label='Frequency',
    scale=y_sc
)

y_ax = bqp.Axis(
#     label='',
    scale=x_ord,
    orientation='vertical'
)

# function that the data frame used for plotting depending on the value in 'column' dropdown and 'select year' rangeslider
def select_df(d):
    global df
    
    if years.value[0]==years.value[1]:
        df = temp_df[column.value].loc[temp_df["YearFrom"]==years.value[0]].value_counts().rename_axis('unique_values').reset_index(name='counts')
    else:
        df = temp_df[column.value].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')

        
# update plot function
def set_plot(b):
    Bar.x = df['unique_values']
    Bar.y = df['counts']


#assign widget actions to functions
column.observe(select_df, names = 'value') #the names part tells .observe the change name to look for, this is only looking for value changes
years.observe(select_df, names = 'value')        
b_refresh.on_click(set_plot)

In [6]:
#display all

fig = bqp.Figure(
    layout=Layout(width='1920px', height=str(len(df)*12)+'px', border ='solid 1px gray'),
    axes=[x_ax, y_ax],
    marks=[Bar],
    fig_margin=dict(top=10, bottom=80, left=835, right=20),
    animation_duration=500
)

box = HBox(
    children=(column, years, b_refresh),
    layout=Layout(margin = '10px 0px 10px 0px')
)

vert_box  = VBox(
    children=(box, fig),
    layout=Layout(border='solid 1px gray', margin = '0 0 0 0')
)

display (vert_box)